<a href="https://colab.research.google.com/github/8066-asad/Extracting-Embodied-Carbon-from-Structures-from-IFC-file/blob/main/Extracting_Embodied_Carbon_from_Struture_from_IFC_file_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Enhanced IFC Embodied Carbon Calculator
# Calculates embodied carbon from IFC building models using carbon factor database

import ifcopenshell
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple
import logging
from pathlib import Path

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class IFCCarbonCalculator:
    """
    A comprehensive tool for calculating embodied carbon from IFC models
    """

    def __init__(self, carbon_factors_path: str):
        """
        Initialize the calculator with carbon factors database

        Args:
            carbon_factors_path: Path to CSV file containing carbon factors
        """
        self.carbon_db = pd.read_csv(carbon_factors_path)
        self.ifc_file = None
        self.results = []

        # Enhanced element types with more comprehensive mapping
        self.element_mapping = {
            "IfcWall": {"material": "concrete", "density": 2400, "unit_preference": "m3"},
            "IfcSlab": {"material": "concrete", "density": 2400, "unit_preference": "m3"},
            "IfcBeam": {"material": "steel", "density": 7850, "unit_preference": "kg"},
            "IfcColumn": {"material": "steel", "density": 7850, "unit_preference": "kg"},
            "IfcRoof": {"material": "concrete", "density": 2400, "unit_preference": "m3"},
            "IfcStair": {"material": "concrete", "density": 2400, "unit_preference": "m3"},
            "IfcRailing": {"material": "steel", "density": 7850, "unit_preference": "kg"},
            "IfcDoor": {"material": "timber", "density": 600, "unit_preference": "kg"},
            "IfcWindow": {"material": "aluminum", "density": 2700, "unit_preference": "kg"},
            "IfcBuildingElementProxy": {"material": "concrete", "density": 2400, "unit_preference": "m3"}
        }

        logger.info(f"Carbon factors database loaded with {len(self.carbon_db)} entries")

    def load_ifc_file(self, ifc_path: str):
        """
        Load IFC file for processing

        Args:
            ifc_path: Path to IFC file
        """
        try:
            self.ifc_file = ifcopenshell.open(ifc_path)
            logger.info(f"IFC file loaded successfully: {ifc_path}")

            # Get basic project information
            project = self.ifc_file.by_type("IfcProject")[0] if self.ifc_file.by_type("IfcProject") else None
            if project:
                logger.info(f"Project: {project.Name}")

        except Exception as e:
            logger.error(f"Error loading IFC file: {e}")
            raise

    def get_element_properties(self, element) -> Dict:
        """
        Extract properties from IFC element with improved error handling

        Args:
            element: IFC element

        Returns:
            Dictionary of element properties
        """
        properties = {
            "NetVolume": 0.0,
            "GrossVolume": 0.0,
            "NetArea": 0.0,
            "GrossArea": 0.0,
            "Length": 0.0,
            "Width": 0.0,
            "Height": 0.0
        }

        try:
            # Check property sets
            if hasattr(element, 'IsDefinedBy'):
                for definition in element.IsDefinedBy:
                    if hasattr(definition, "RelatingPropertyDefinition"):
                        props = definition.RelatingPropertyDefinition
                        if hasattr(props, "HasProperties"):
                            for prop in props.HasProperties:
                                if prop.Name in properties:
                                    try:
                                        value = float(prop.NominalValue.wrappedValue)
                                        properties[prop.Name] = value
                                    except:
                                        continue

            # Try to get quantities from quantity sets
            if hasattr(element, 'IsDefinedBy'):
                for definition in element.IsDefinedBy:
                    if hasattr(definition, "RelatingPropertyDefinition"):
                        if definition.RelatingPropertyDefinition.is_a("IfcElementQuantity"):
                            for quantity in definition.RelatingPropertyDefinition.Quantities:
                                if quantity.Name in properties:
                                    try:
                                        if hasattr(quantity, 'VolumeValue'):
                                            properties[quantity.Name] = float(quantity.VolumeValue)
                                        elif hasattr(quantity, 'AreaValue'):
                                            properties[quantity.Name] = float(quantity.AreaValue)
                                        elif hasattr(quantity, 'LengthValue'):
                                            properties[quantity.Name] = float(quantity.LengthValue)
                                    except:
                                        continue

        except Exception as e:
            logger.warning(f"Error extracting properties for element {element.GlobalId}: {e}")

        return properties

    def get_material_info(self, element) -> Tuple[str, float]:
        """
        Extract material information from IFC element

        Args:
            element: IFC element

        Returns:
            Tuple of (material_name, density)
        """
        material_name = "unknown"
        density = 0.0

        try:
            # Try to get material from material associations
            if hasattr(element, 'HasAssociations'):
                for association in element.HasAssociations:
                    if association.is_a("IfcRelAssociatesMaterial"):
                        material = association.RelatingMaterial
                        if hasattr(material, 'Name') and material.Name:
                            material_name = material.Name.lower()
                            # Try to match with known materials
                            if any(mat in material_name for mat in ['concrete', 'cement']):
                                density = 2400
                            elif any(mat in material_name for mat in ['steel', 'iron']):
                                density = 7850
                            elif any(mat in material_name for mat in ['timber', 'wood']):
                                density = 600
                            elif any(mat in material_name for mat in ['aluminum', 'aluminium']):
                                density = 2700
                            break
        except Exception as e:
            logger.warning(f"Error extracting material for element {element.GlobalId}: {e}")

        return material_name, density

    def calculate_quantity(self, element, ifc_type: str) -> Tuple[float, str]:
        """
        Calculate quantity for the element based on available data

        Args:
            element: IFC element
            ifc_type: IFC type string

        Returns:
            Tuple of (quantity, unit)
        """
        properties = self.get_element_properties(element)
        element_config = self.element_mapping.get(ifc_type, {})

        # Get volume (prioritize NetVolume, then GrossVolume)
        volume = properties.get("NetVolume", 0.0)
        if volume <= 0:
            volume = properties.get("GrossVolume", 0.0)

        # If no volume, try to calculate from dimensions
        if volume <= 0:
            length = properties.get("Length", 0.0)
            width = properties.get("Width", 0.0)
            height = properties.get("Height", 0.0)

            if length > 0 and width > 0 and height > 0:
                volume = length * width * height

        # Determine unit and quantity based on element type
        preferred_unit = element_config.get("unit_preference", "m3")
        density = element_config.get("density", 2400)

        if preferred_unit == "kg" and volume > 0:
            return volume * density, "kg"
        elif volume > 0:
            return volume, "m3"
        else:
            return 0.0, preferred_unit

    def find_carbon_factor(self, ifc_type: str, material: str, unit: str) -> float:
        """
        Find carbon factor from database with fuzzy matching

        Args:
            ifc_type: IFC type
            material: Material name
            unit: Unit of measurement

        Returns:
            Carbon factor value
        """
        # Try exact match first
        exact_match = self.carbon_db[
            (self.carbon_db["ifc_type"] == ifc_type) &
            (self.carbon_db["material"].str.lower() == material.lower()) &
            (self.carbon_db["unit"] == unit)
        ]

        if not exact_match.empty:
            return float(exact_match.iloc[0]["carbon_factor (kgCO2e/unit)"])

        # Try material and unit match
        material_match = self.carbon_db[
            (self.carbon_db["material"].str.lower() == material.lower()) &
            (self.carbon_db["unit"] == unit)
        ]

        if not material_match.empty:
            return float(material_match.iloc[0]["carbon_factor (kgCO2e/unit)"])

        # Try material match only
        material_only = self.carbon_db[
            self.carbon_db["material"].str.lower() == material.lower()
        ]

        if not material_only.empty:
            return float(material_only.iloc[0]["carbon_factor (kgCO2e/unit)"])

        logger.warning(f"No carbon factor found for IFC Type: {ifc_type}, Material: {material}, Unit: {unit}")
        return 0.0

    def calculate_embodied_carbon(self, element_types: Optional[List[str]] = None) -> pd.DataFrame:
        """
        Calculate embodied carbon for all or specified element types

        Args:
            element_types: List of IFC types to process. If None, process all mapped types

        Returns:
            DataFrame with results
        """
        if not self.ifc_file:
            raise ValueError("No IFC file loaded. Call load_ifc_file() first.")

        if element_types is None:
            element_types = list(self.element_mapping.keys())

        results = []

        for ifc_type in element_types:
            logger.info(f"Processing {ifc_type} elements...")

            elements = self.ifc_file.by_type(ifc_type)
            logger.info(f"Found {len(elements)} {ifc_type} elements")

            for element in elements:
                try:
                    # Get quantity and unit
                    quantity, unit = self.calculate_quantity(element, ifc_type)
                    logger.debug(f"Element {element.GlobalId}: Quantity {quantity}, Unit {unit}")

                    # Get material info
                    material_from_element, density_from_element = self.get_material_info(element)

                    # Use detected material or fallback to mapping
                    if material_from_element != "unknown":
                        material = material_from_element
                    else:
                        material = self.element_mapping.get(ifc_type, {}).get("material", "unknown")

                    logger.debug(f"Element {element.GlobalId}: Detected Material {material_from_element}, Used Material {material}")

                    # Find carbon factor
                    carbon_factor = self.find_carbon_factor(ifc_type, material, unit)
                    logger.debug(f"Element {element.GlobalId}: Carbon Factor {carbon_factor}")

                    # Calculate embodied carbon
                    embodied_carbon = quantity * carbon_factor

                    # Store results
                    results.append({
                        "Element GUID": element.GlobalId,
                        "Element Name": getattr(element, 'Name', 'Unnamed'),
                        "IFC Type": ifc_type,
                        "Material": material,
                        "Quantity Used": round(quantity, 3),
                        "Unit": unit,
                        "Carbon Factor (kgCO2e/unit)": carbon_factor,
                        "Embodied Carbon (kgCO2e)": round(embodied_carbon, 2)
                    })

                except Exception as e:
                    logger.error(f"Error processing element {element.GlobalId}: {e}")
                    continue

        self.results = results
        return pd.DataFrame(results)

    def generate_summary_report(self, df: pd.DataFrame) -> Dict:
        """
        Generate summary statistics from results

        Args:
            df: Results DataFrame

        Returns:
            Dictionary with summary statistics
        """
        summary = {
            "total_elements": len(df),
            "total_embodied_carbon": df["Embodied Carbon (kgCO2e)"].sum(),
            "by_material": df.groupby("Material")["Embodied Carbon (kgCO2e)"].sum().to_dict(),
            "by_ifc_type": df.groupby("IFC Type")["Embodied Carbon (kgCO2e)"].sum().to_dict(),
            "elements_without_carbon_factor": len(df[df["Carbon Factor (kgCO2e/unit)"] == 0]),
        }

        return summary

    def export_results(self, df: pd.DataFrame, output_path: str = "carbon_report.csv"):
        """
        Export results to CSV with summary

        Args:
            df: Results DataFrame
            output_path: Output file path
        """
        # Export detailed results
        df.to_csv(output_path, index=False)

        # Generate and export summary
        summary = self.generate_summary_report(df)
        summary_path = output_path.replace('.csv', '_summary.csv')

        summary_df = pd.DataFrame([
            {"Metric": "Total Elements", "Value": summary["total_elements"]},
            {"Metric": "Total Embodied Carbon (kgCO2e)", "Value": f"{summary['total_embodied_carbon']:.2f}"},
            {"Metric": "Elements Without Carbon Factor", "Value": summary["elements_without_carbon_factor"]},
        ])

        summary_df.to_csv(summary_path, index=False)

        logger.info(f"Results exported to {output_path}")
        logger.info(f"Summary exported to {summary_path}")

        # Print summary to console
        print("\n" + "="*60)
        print("EMBODIED CARBON CALCULATION SUMMARY")
        print("="*60)
        print(f"Total Elements Processed: {summary['total_elements']}")
        print(f"Total Embodied Carbon: {summary['total_embodied_carbon']:.2f} kgCO2e")
        print(f"Elements Without Carbon Factor: {summary['elements_without_carbon_factor']}")

        print("\nEmbodied Carbon by Material:")
        for material, carbon in summary['by_material'].items():
            print(f"  {material}: {carbon:.2f} kgCO2e")

        print("\nEmbodied Carbon by IFC Type:")
        for ifc_type, carbon in summary['by_ifc_type'].items():
            print(f"  {ifc_type}: {carbon:.2f} kgCO2e")
        print("="*60)


def main():
    """
    Main execution function for use in Google Colab or standalone
    """
    # For Google Colab
    try:
        from google.colab import files

        print("📤 Please upload your IFC model file (.ifc)")
        uploaded = files.upload()
        ifc_filename = [f for f in uploaded.keys() if f.lower().endswith('.ifc')][0]

        print("📤 Please upload your carbon_factors.csv file")
        uploaded_csv = files.upload()
        csv_filename = [f for f in uploaded_csv.keys() if f.lower().endswith('.csv')][0]

    except ImportError:
        # For standalone use
        print("Enter the path to your IFC file:")
        ifc_filename = input().strip()
        print("Enter the path to your carbon factors CSV file:")
        csv_filename = input().strip()

    try:
        # Initialize calculator
        calculator = IFCCarbonCalculator(csv_filename)

        # Load IFC file
        calculator.load_ifc_file(ifc_filename)

        # Calculate embodied carbon
        results_df = calculator.calculate_embodied_carbon()

        # Export results
        calculator.export_results(results_df)

        # Download files in Colab
        try:
            files.download("carbon_report.csv")
            files.download("carbon_report_summary.csv")
        except:
            pass

    except Exception as e:
        logger.error(f"Error in main execution: {e}")
        raise


if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.3 MB/s eta 0:00:00
📤 Please upload your IFC model file (.ifc)


Saving Revit Model_806622EU3.ifc to Revit Model_806622EU3.ifc
📤 Please upload your carbon_factors.csv file


Saving carbon_factors.csv to carbon_factors.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>